# MODIS data dowloading and processing

## Required python libraries

glob, requests, gdal (gdal installation: brew install gdal, which does not include hdf4 driver)


## Building hdf4 driver on mac

1. get hdf-4.2.10.tar.gz (https://support.hdfgroup.org/ftp/HDF/releases/HDF4.2.10/src/hdf-4.2.10.tar.gz)
2. unzip hdf-4.2.10.tar.gz and go into the directory
3. cd hdf-4.2.10 && ./configure --disable-fortran --enable-production --enable-shared --disable-netcdf --with-zlib=/usr --with-jpeg=/usr/local --prefix=/usr/local/hdf4
4. make >& make.out
5. make check >& check.out
6. make install


## Building pyhdf4 wrapper on mac

1. export INCLUDE_DIRS=/usr/local/hdf4/include (make sure consist with the hdf configuration)
2. export LIBRARY_DIRS=/usr/local/hdf4/lib (make sure consist with the hdf configuration)
3. pip3 install pyhdf


## Utility file - maskmodis.py

The file was modified from pymasker (hdf4 file was read by pyhdf4) and the copyright should belong to the original authors. Specific procedure of QA masking, please read https://stevemosher.wordpress.com/2012/12/05/modis-qc-bits/


## Utility file - downmodis.py

The file was modified from pyModis.downmodis and the copyright should belong to the original authors

In [171]:
# define a class for processing modis 1km LST

from pyhdf.SD import SD, SDC
from utils import parsemodis
from utils.maskmodis import ModisQuality, Masker
import matplotlib.pyplot as plt
import earthpy.plot as ep

class getModisLST_1km():
    
    def __init__(self, MODIS_file, prefix, lats, longs):
        
        self.MODIS_file = MODIS_file
        self.lats = lats
        self.longs = longs
        self.modisParser = parsemodis.parseModis(MODIS_file)
        self.prefix = prefix
        
        quality = ModisQuality.high # set quality to high
        file = SD(MODIS_file, SDC.READ) # open hdf4 file
        # read data according prefix
        if prefix == 'Day':
            # get day time temperature
            self.data = file.select('LST_Day_1km').get()
            masker = Masker(band=MODIS_file, band_name='QC_Day')
        elif prefix == 'Night':
            # get night time temperature
            self.data = file.select('LST_Night_1km').get()
            masker = Masker(band=MODIS_file, band_name='QC_Night')
        else:
            print('Prefix can only be Day or Night')
            return(0)
        self.mask = masker.get_mask(0, 2, quality).astype(int) # QA mask
        
        
    def printBands(self):
        file = SD(self.MODIS_file, SDC.READ)
        datasets_dic = file.datasets()
        for idx,sds in enumerate(datasets_dic.keys()):
            print(idx,sds)
        
        
    def runQC(self):
        return self.data * self.mask
    
    
    def plotQC(self):
        plt.figure(figsize=(20,10))
        im = plt.imshow(self.mask)
        ep.colorbar(im)
        plt.show()
        
        
    def plotData(self):
        plt.figure(figsize=(20,10))
        im = plt.imshow(self.data)
        ep.colorbar(im)
        plt.show()
    
    
    def toCelsius(self, modis_data):
        modis_data = (modis_data * 1.0) # convert into float
        modis_data[np.where(modis_data == 0)] = np.nan # set invalided value to nan
        modis_data = modis_data * 0.02 - 273.15 # convert to celsius
        return modis_data
            

    def getVal(self, modis_data):
        # get geoinformation
        bound = self.modisParser.retBoundary()
        long_res = (bound['max_lon'] - bound['min_lon']) / self.data.shape[1] # col - x
        lat_res = (bound['max_lat'] - bound['min_lat']) / self.data.shape[0] # row - y

        # retreive value based on lat/long
        rows = ((bound['max_lat'] - np.array(lats)) / lat_res).astype(int)
        cols = ((np.array(longs) - bound['min_lon']) / long_res).astype(int)
        
        # return value from modis_data
        return [modis_data[(row, col)] for row, col in zip(rows, cols)]
    
    
    def createVarName(self):
        # define name of the column
        time_range = self.modisParser.retRangeTime()
        return self.prefix+'_'+time_range['RangeBeginningDate']+'_'+time_range['RangeEndingDate']
    
    
    def getLST_QC(self):
        modis_data = self.runQC()
        modis_data = self.toCelsius(modis_data)
        return self.getVal(modis_data)
        
        
    def getLST(self):
        modis_data = self.toCelsius(self.data)
        return self.getVal(modis_data)
        

In [15]:
# import downmodis, which is modified from pyMODIS code
from utilities import downmodis

In [13]:
# Variables for data download
dest = "data/" # This directory must already exist BTW
tiles = "h11v04" # That's the MODIS tile covering northern Europe
day = "2016.06.01"
enddate = "2016.09.30" # The download works backward, so that enddate is anterior to day=
product = "MOD11A2.006"

# Instantiate download class, connect and download
modis_down = downmodis.downModis(destinationFolder=dest, tiles=tiles, today=day, enddate=enddate, product=product)
modis_down.connect()
modis_down.downloadsAllDay()

# Check that the data has been downloaded
MODIS_files = glob.glob(dest + '*.hdf')
print(MODIS_files)

['data/MOD11A2.A2016161.h11v04.006.2016242164334.hdf', 'data/MOD11A2.A2016265.h11v04.006.2016274190024.hdf', 'data/MOD11A2.A2016177.h11v04.006.2016242171048.hdf', 'data/MOD11A2.A2016225.h11v04.006.2016244043027.hdf', 'data/MOD11A2.A2016185.h11v04.006.2016242172459.hdf', 'data/MOD11A2.A2016257.h11v04.006.2016267020253.hdf', 'data/MOD11A2.A2016209.h11v04.006.2016243094426.hdf', 'data/MOD11A2.A2016217.h11v04.006.2016243195226.hdf', 'data/MOD11A2.A2016193.h11v04.006.2016242175058.hdf', 'data/MOD11A2.A2016241.h11v04.006.2016252204234.hdf', 'data/MOD11A2.A2016169.h11v04.006.2016242165624.hdf', 'data/MOD11A2.A2016153.h11v04.006.2016242163427.hdf', 'data/MOD11A2.A2016249.h11v04.006.2016258072147.hdf', 'data/MOD11A2.A2016233.h11v04.006.2016252174937.hdf', 'data/MOD11A2.A2016201.h11v04.006.2016242234119.hdf', 'data/MOD11A2.A2016273.h11v04.006.2016282072743.hdf']


In [173]:
# extract value from all the hdfs
import pandas as pd


# read file, extract lats and longs, then using the loop to extract the values

for iFile in MODIS_files:
    modisLST = getModisLST_1km(MODIS_files[0], 'Day', lats, longs)
    var_name = modisLST.createVarName()
    modisLST.getLST()

[35.710000000000036, 19.890000000000043]